# 3D Plotting 🌪️

In [ ]:
import numpy as np
import torch
import warnings
from plotly.colors import qualitative
import plotly.graph_objects as go
from plotly.colors import qualitative
import warnings
from optimizers.adam_clara import Adam_CLARA
from optimizers.sgd_clara import SGD_CLARA
import webbrowser
import os

----
### Mathematical Functions 🧮

In [ ]:
def sphere(x):
    return np.sum(x**2)

def sphere_torch(x):
    return torch.sum(x**2)

sigma = 0.1  # Noise std
def sphere_noisy(x, sigma=sigma):
    x_noisy = x + np.random.normal(0, sigma, size=x.shape)
    return np.sum(x_noisy**2)

def sphere_noisy_torch(x, sigma=sigma):
    x_noisy = x + sigma * torch.randn_like(x)
    return torch.sum(x_noisy**2)

cond = 1e3
def ellipsoid(x):
    n = len(x)
    weights = np.power(cond, np.arange(n) / (n - 1))
    return np.sum(weights * x**2)

def ellipsoid_torch(x):
    n = len(x)
    weights = torch.pow(torch.tensor(cond), torch.arange(n, dtype=x.dtype, device=x.device) / (n - 1))
    return torch.sum(weights * x**2)

def ellipsoid_noisy(x, sigma=sigma):
    x_noisy = x + np.random.normal(0, sigma, size=x.shape)
    n = len(x_noisy)
    weights = np.power(cond, np.arange(n) / (n - 1))
    return np.sum(weights * x_noisy**2)

def ellipsoid_noisy_torch(x, sigma=sigma):
    n = len(x)
    weights = torch.pow(torch.tensor(cond, dtype=x.dtype, device=x.device),
                        torch.arange(n, dtype=x.dtype, device=x.device) / (n - 1))
    x_noisy = x + sigma * torch.randn_like(x)
    return torch.sum(weights * x_noisy**2)

def rosenbrock(x):
    a, b = 1, 100
    return (a - x[0])**2 + b*(x[1] - x[0]**2)**2

def rosenbrock_torch(x):
    a, b = 1, 100
    return (a - x[0])**2 + b*(x[1] - x[0]**2)**2


def himmelblau(x):
    return (x[0]**2 + x[1] - 11)**2 + (x[0] + x[1]**2 - 7)**2

def himmelblau_torch(x):
    return (x[0]**2 + x[1] - 11)**2 + (x[0] + x[1]**2 - 7)**2


def beale(x):
    return ((1.5 - x[0] + x[0]*x[1])**2 +
            (2.25 - x[0] + x[0]*x[1]**2)**2 +
            (2.625 - x[0] + x[0]*x[1]**3)**2)

def beale_torch(x):
    return ((1.5 - x[0] + x[0]*x[1])**2 +
            (2.25 - x[0] + x[0]*x[1]**2)**2 +
            (2.625 - x[0] + x[0]*x[1]**3)**2)
    
    
def eggholder(x):
    return (-(x[1]+47)*np.sin(np.sqrt(abs(x[0]/2 + (x[1]+47)))) - 
            x[0]*np.sin(np.sqrt(abs(x[0] - (x[1]+47)))))

def eggholder_torch(x):
    term1 = -(x[1]+47)*torch.sin(torch.sqrt(torch.abs(x[0]/2 + (x[1]+47))))
    term2 = -x[0]*torch.sin(torch.sqrt(torch.abs(x[0] - (x[1]+47))))
    return term1 + term2


def three_hump_camel(x):
    return 2*x[0]**2 - 1.05*x[0]**4 + (x[0]**6)/6 + x[0]*x[1] + x[1]**2

def three_hump_camel_torch(x):
    return (2*x[0]**2 - 1.05*x[0]**4 + (x[0]**6)/6 + x[0]*x[1] + x[1]**2)


def ackley(x):
    x0, x1 = x[0], x[1]
    part1 = -20*np.exp(-0.2*np.sqrt(0.5*(x0**2 + x1**2)))
    part2 = -np.exp(0.5*(np.cos(2*np.pi*x0) + np.cos(2*np.pi*x1)))
    return part1 + part2 + np.e + 20

def ackley_torch(x):
    part1 = -20*torch.exp(-0.2*torch.sqrt(0.5*(x[0]**2 + x[1]**2)))
    part2 = -torch.exp(0.5*(torch.cos(2*torch.pi*x[0]) + torch.cos(2*torch.pi*x[1])))
    return part1 + part2 + torch.exp(torch.tensor(1.0)) + 20

----
### Helper Functions 🛠️

In [ ]:
def to_torch_tensor(x, device='cpu'):
    return torch.tensor(x, dtype=torch.float32, device=device, requires_grad=True)


def numpy_func_wrapper(func):
    """Return a function that converts torch.Tensor to numpy and applies func"""
    def f(x_torch):
        x_np = x_torch.detach().cpu().numpy()
        return func(x_np)
    return f

# %%
def get_func_and_torch(name):
    funcs_np = {
        "sphere": sphere,
        "sphere_noisy": sphere_noisy,
        "ellipsoid": ellipsoid,
        "ellipsoid_noisy": ellipsoid_noisy,
        "rosenbrock": rosenbrock,
        "himmelblau": himmelblau,
        "beale": beale,
        "eggholder": eggholder,
        "three_hump_camel": three_hump_camel,
        "ackley": ackley,
    }
    funcs_torch = {
        "sphere": sphere_torch,
        "sphere_noisy": sphere_noisy_torch,
        "ellipsoid": ellipsoid_torch,
        "ellipsoid_noisy": ellipsoid_noisy_torch,
        "rosenbrock": rosenbrock_torch,
        "himmelblau": himmelblau_torch,
        "beale": beale_torch,
        "eggholder": eggholder_torch,
        "three_hump_camel": three_hump_camel_torch,
        "ackley": ackley_torch,
    }
    return funcs_np[name], funcs_torch[name]

----
### Plot- und Optimierungsmethode 📊

In [ ]:
def show_plot_and_save(fig, permanent_path="./results/", filename=None):
    """Create an HTML file with a custom name, save it, and open it."""
    os.makedirs(permanent_path, exist_ok=True)
    file_path = os.path.join(permanent_path, filename)
    fig.write_html(file_path, auto_open=False)
    print(f"✅ Plot saved as {file_path}")


def optimize_and_plot(
    func_np, func_torch, global_optimum,
    optimizers,
    start_points,
    bounds=[-2, 2],
    steps_dict=None,
    lr_dict=None,
    gradient_clip=None,
    device='cpu',
    func_name=None,
    popup=False,  # Neues Argument für optionales Popup
    clip_surface=False
):
    # Surface grid
    x_vals = y_vals = np.linspace(bounds[0], bounds[1], 200)
    X, Y = np.meshgrid(x_vals, y_vals)
    Z = np.zeros_like(X)

    # Compute surface
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            try:
                val = func_np(np.array([X[i, j], Y[i, j]]))
                Z[i, j] = val if np.isfinite(val) else np.nan
            except Exception:
                Z[i, j] = np.nan

    print(f"[{func_name}] Z-statistics before clipping: min={np.nanmin(Z):.2e}, max={np.nanmax(Z):.2e}")

    # ------------------------------------------------
    zmin, zmax = np.nanmin(Z), np.nanmax(Z)
    print(f"[{func_name}] Clipping disabled – Using full Z-range: {zmin:.2f} to {zmax:.2f}")
    # ------------------------------------------------

    # Optimization paths
    paths = {}
    color_cycle = qualitative.Plotly
    color_list = list(color_cycle)
    color_map = {}

    for idx, (name, OptimizerClass) in enumerate(optimizers.items()):
        lr = lr_dict.get(name, 0.01) if lr_dict else 0.01
        steps = steps_dict.get(name, 50) if steps_dict else 50
        start = start_points[name]

        x_torch = torch.tensor(start, dtype=torch.float32, device=device, requires_grad=True)
        optimizer = OptimizerClass([x_torch], lr=lr)
        path = [x_torch.detach().cpu().clone().numpy()]

        for step_idx in range(steps):
            optimizer.zero_grad()
            val = func_torch(x_torch)
            val.backward()

            if gradient_clip:
                torch.nn.utils.clip_grad_norm_([x_torch], max_norm=gradient_clip)

            optimizer.step()

            with torch.no_grad():
                x_torch[:] = torch.clamp(x_torch, bounds[0], bounds[1])

            if not torch.isfinite(x_torch).all() or (x_torch.abs() > 1e3).any():
                print(f"⚠️ Divergenz in Optimierer '{name}' bei Schritt {step_idx}: {x_torch.detach().cpu().numpy()}")
                break

            path.append(x_torch.detach().cpu().clone().numpy())

        paths[name] = np.array(path)
        color_map[name] = color_list[idx % len(color_list)]

    # Plotly figure
    fig = go.Figure()

    fig.add_trace(go.Surface(
        x=X, y=Y, z=Z,
        colorscale='Viridis',
        opacity=0.9,
        name='Loss Surface',
        showscale=True,
        colorbar=dict(title='Loss')
    ))

    for name, path in paths.items():
        try:
            Z_path = [np.clip(func_np(p), zmin, zmax) for p in path]
        except Exception as e:
            warnings.warn(f"Pfadberechnung für {name} fehlgeschlagen: {e}")
            Z_path = [zmax for _ in path]

        fig.add_trace(go.Scatter3d(
            x=path[:, 0], y=path[:, 1], z=Z_path,
            mode='lines+markers',
            line=dict(color=color_map[name], width=4),
            marker=dict(size=3),
            name=f'{name} Pfad'
        ))

    fig.add_trace(go.Scatter3d(
        x=[global_optimum[0]],
        y=[global_optimum[1]],
        z=[np.clip(func_np(global_optimum), zmin, zmax)],
        mode='markers+text',
        marker=dict(size=8, color='purple'),
        text=["Globales Optimum"],
        textposition='top center',
        name='Globales Optimum'
    ))

    fig.update_layout(
        title=f"Optimierungspfade auf Zielfunktion: {func_name}",
        autosize=True,
        scene=dict(
            xaxis_title="x",
            yaxis_title="y",
            zaxis_title="f(x, y)",
            zaxis=dict(range=[zmin, zmax]),
            camera=dict(eye=dict(x=1.5, y=1.5, z=1.0))
        ),
        margin=dict(l=0, r=0, t=50, b=0),
        font=dict(family="Arial", size=14),
        legend=dict(x=0.8, y=0.95)
    )


    fig.show()
    show_plot_and_save(fig, permanent_path="./results/", filename=f"{func_name}_optimization_paths.html")

In [ ]:
def optimize_and_plot_2d(
    func_np, func_torch, global_optimum,
    optimizers,
    start_points,
    bounds=[-2, 2],
    steps_dict=None,
    lr_dict=None,
    gradient_clip=None,
    device='cpu',
    func_name=None,
    popup=False,
    clip_surface=False
):
    # Surface grid
    x_vals = y_vals = np.linspace(bounds[0], bounds[1], 200)
    X, Y = np.meshgrid(x_vals, y_vals)
    Z = np.zeros_like(X)

    # Compute surface
    for i in range(X.shape[0]):
        for j in range(X.shape[1]):
            try:
                val = func_np(np.array([X[i, j], Y[i, j]]))
                Z[i, j] = val if np.isfinite(val) else np.nan
            except Exception:
                Z[i, j] = np.nan

    print(f"[{func_name}] Z-statistics before clipping: min={np.nanmin(Z):.2e}, max={np.nanmax(Z):.2e}")

    zmin, zmax = np.nanmin(Z), np.nanmax(Z)
    print(f"[{func_name}] Clipping disabled – Using full Z-range: {zmin:.2f} to {zmax:.2f}")

    # Optimization paths
    paths = {}
    color_cycle = qualitative.Plotly
    color_list = list(color_cycle)
    color_map = {}

    for idx, (name, OptimizerClass) in enumerate(optimizers.items()):
        lr = lr_dict.get(name, 0.01) if lr_dict else 0.01
        steps = steps_dict.get(name, 50) if steps_dict else 50
        start = start_points[name]

        x_torch = torch.tensor(start, dtype=torch.float32, device=device, requires_grad=True)
        optimizer = OptimizerClass([x_torch], lr=lr)
        path = [x_torch.detach().cpu().clone().numpy()]

        for step_idx in range(steps):
            optimizer.zero_grad()
            val = func_torch(x_torch)
            val.backward()

            if gradient_clip:
                torch.nn.utils.clip_grad_norm_([x_torch], max_norm=gradient_clip)

            optimizer.step()

            with torch.no_grad():
                x_torch[:] = torch.clamp(x_torch, bounds[0], bounds[1])

            if not torch.isfinite(x_torch).all() or (x_torch.abs() > 1e3).any():
                print(f"⚠️ Divergenz in Optimierer '{name}' bei Schritt {step_idx}: {x_torch.detach().cpu().numpy()}")
                break

            path.append(x_torch.detach().cpu().clone().numpy())

        paths[name] = np.array(path)
        color_map[name] = color_list[idx % len(color_list)]

    # Plotly 2D Contour Figure
    fig = go.Figure()

    fig.add_trace(go.Contour(
        x=x_vals,
        y=y_vals,
        z=Z,
        colorscale='Viridis',
        contours=dict(showlabels=True),
        colorbar=dict(title="f(x, y)"),
        line_smoothing=0.5,
        zmin=zmin,
        zmax=zmax
    ))

    for name, path in paths.items():
        fig.add_trace(go.Scatter(
            x=path[:, 0], y=path[:, 1],
            mode='lines+markers',
            line=dict(color=color_map[name], width=3),
            marker=dict(size=5),
            name=f'{name} Pfad'
        ))

    # Global Optimum
    fig.add_trace(go.Scatter(
        x=[global_optimum[0]],
        y=[global_optimum[1]],
        mode='markers+text',
        marker=dict(size=10, color='purple'),
        text=["Globales Optimum"],
        textposition='top center',
        name='Globales Optimum'
    ))

    fig.update_layout(
        title=f"Optimierungspfade auf Zielfunktion: {func_name}",
        autosize=True,
        scene=dict(
            xaxis_title="x",
            yaxis_title="y",
            zaxis_title="f(x, y)",
            zaxis=dict(range=[zmin, zmax]),
            camera=dict(eye=dict(x=1.5, y=1.5, z=1.0))
        ),
        margin=dict(l=0, r=0, t=50, b=0),
        font=dict(family="Arial", size=14),
        legend=dict(x=0.8, y=0.95)
    )

    fig.show()
    show_plot_and_save(fig, permanent_path="./results/", filename=f"{func_name}_optimization_paths_2d.html")


-----
### Main Function 🚀

In [ ]:
if __name__ == "__main__":
    # Optimizer Klassen
    optimizers = {
        # 'SGD': torch.optim.SGD,
        'ADAM': torch.optim.Adam,
        'Adam_CLARA': Adam_CLARA,
        # 'SGD_CLARA': SGD_CLARA,
    }

    # Funktion-spezifische Startpunkte
    per_func_startpoints = {
        "sphere": {
            'SGD': np.array([1.0, 1.0]),
            'ADAM': np.array([1.0, 1.0]),
            'Adam_CLARA': np.array([-1.0, -1.0]),
            'SGD_CLARA': np.array([1.0, 1.0])
        },
        "sphere_noisy": {
            'SGD': np.array([1.0, 1.0]),
            'ADAM': np.array([1.0, 1.0]),
            'Adam_CLARA': np.array([-1.0, -1.0]),
            'SGD_CLARA': np.array([1.0, 1.0])
        },
        "ellipsoid": {
            'SGD': np.array([1.0, 1.0]),
            'ADAM': np.array([1.0, 1.0]),
            'Adam_CLARA': np.array([-1.0, -1.0]),
            'SGD_CLARA': np.array([1.0, 1.0])
        },
        "ellipsoid_noisy": {
            'SGD': np.array([1.0, 1.0]),
            'ADAM': np.array([1.0, 1.0]),
            'Adam_CLARA': np.array([-1.0, -1.0]),
            'SGD_CLARA': np.array([1.0, 1.0])
        },
        "rosenbrock": {
            'SGD': np.array([-1.0, 1.0]),  # np.array([-1.5, 1.5]),
            'ADAM': np.array([-1.0, 1.0]),  # np.array([1.5, -1.5]),
            'Adam_CLARA': np.array([-1.0, 1.0]),
            'SGD_CLARA': np.array([-1.0, 1.0]),  # np.array([1.0, -1.0]),
        },
        "himmelblau": {
            'SGD': np.array([-4.0, 4.0]),
            'ADAM': np.array([-4.0, 4.0]),  # np.array([4.0, -4.0]),
            'Adam_CLARA': np.array([-4.0, 4.0]),  # np.array([-3.0, 3.0]),
            'SGD_CLARA': np.array([-4.0, 4.0])  # np.array([3.0, -3.0]),
        },
        "beale": {
            'SGD': np.array([1.0, 1.0]),
            'ADAM': np.array([1.0, 1.0]),  # np.array([2.5, 2.0]),
            'Adam_CLARA': np.array([1.0, 1.0]),  # np.array([-1.0, 1.0]),
            'SGD_CLARA': np.array([1.0, 1.0])  # np.array([1.5, -1.5]),
        },
        "eggholder": {
            'SGD': np.array([500, 300]),
            'ADAM': np.array([500, 300]),  # np.array([300, 500]),
            'Adam_CLARA': np.array([500, 300]),  # np.array([400, 400]),
            'SGD_CLARA': np.array([500, 300])  # np.array([450, 450]),
        },
        "three_hump_camel": {
            'SGD': np.array([1.0, 1.0]),
            'ADAM': np.array([1.0, 1.0]),  # np.array([-1.0, -1.0]),
            'Adam_CLARA': np.array([1.0, 1.0]),  # np.array([1.5, -1.5]),
            'SGD_CLARA': np.array([1.0, 1.0])  # np.array([-1.5, 1.5]),
        },
        "ackley": {
            'SGD': np.array([2.0, 2.0]),
            'ADAM': np.array([2.0, 2.0]),  # np.array([-2.0, -2.0]),
            'Adam_CLARA': np.array([2.0, 2.0]),  # np.array([2.5, -2.5]),
            'SGD_CLARA': np.array([2.0, 2.0])  # np.array([-2.5, 2.5]),
        }
    }

    # Schrittanzahl pro Optimizer
    steps_dict = {
        'SGD': 100,
        'ADAM': 20,
        'Adam_CLARA': 20,
        'SGD_CLARA': 100,
    }

    # Lernraten pro Optimizer
    lr = 1e2  # 1e2
    lr_dict = {
        'SGD': lr,  # 0.01,
        'ADAM': lr,  # 0.005,
        'Adam_CLARA': lr,  # 0.005,
        'SGD_CLARA': lr  # 0.01,
    }

    # Globales Optimum
    global_optima = {
        "sphere": np.array([0.0, 0.0]),
        "sphere_noisy": np.array([0.0, 0.0]),
        "ellipsoid": np.array([0.0, 0.0]),
        "ellipsoid_noisy": np.array([0.0, 0.0]),
        # "rosenbrock": np.array([1.0, 1.0]),
        # "himmelblau": np.array([3.0, 2.0]),
        # "beale": np.array([3.0, 0.5]),
        # "eggholder": np.array([512.0, 404.2319]),
        # "three_hump_camel": np.array([0.0, 0.0]),
        # "ackley": np.array([0.0, 0.0]),
    }

    # Plotbereiche pro Funktion
    func_bounds = {
        "sphere": [-5, 5],
        "sphere_noisy": [-5, 5],
        "ellipsoid": [-5, 5],
        "ellipsoid_noisy": [-5, 5],
        "rosenbrock": [-2, 2],
        "himmelblau": [-6, 6],
        "beale": [-4.5, 4.5],
        "eggholder": [0, 600],
        "three_hump_camel": [-3, 3],
        "ackley": [-5, 5],
    }

    # Liste der Funktionen
    func_names = list(global_optima.keys())

    for func_name in func_names:
        print(f"\n--- Generating plot for: {func_name} ---")
        func_np, func_torch = get_func_and_torch(func_name)
        
        optimize_and_plot(
            func_np=func_np,
            func_torch=func_torch,
            global_optimum=global_optima[func_name],
            optimizers=optimizers,
            start_points=per_func_startpoints[func_name],
            bounds=func_bounds[func_name],
            steps_dict=steps_dict,
            lr_dict=lr_dict,
            gradient_clip=5.0,  
            func_name=func_name  
        )
                
        optimize_and_plot_2d(
            func_np=func_np,
            func_torch=func_torch,
            global_optimum=global_optima[func_name],
            optimizers=optimizers,
            start_points=per_func_startpoints[func_name],
            bounds=func_bounds[func_name],
            steps_dict=steps_dict,
            lr_dict=lr_dict,
            gradient_clip=5.0,  
            func_name=func_name  
        )
